In [ ]:
import pandas as pd

# Load pickle file into dataframe
path = r"C:\Users\viet-intel\boligpriser\data\output\DAR.pkl"

DAR = pd.read_pickle(path)

print(list(DAR.columns))
print(len(DAR))

# select the columns id
bfe_id = DAR[['id_lokalId']]

# adding empty column bfe to the dataframe
bfe_id['BFE'] = ""

display(bfe_id.head())
len(bfe_id)

import requests
import pandas as pd

def get_data_from_api(adresse_ids):
    url = f"https://services.datafordeler.dk/EBR/Ejendomsbeliggenhed/1/rest/BFEnrAdresse?Adresseid={adresse_ids}&username=ANLVUSSNAP&password=Essvietcc567!"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error making the API call:", e)
        return None
    except ValueError as ve:
        print("Error processing JSON response:", ve)
        return None

def chunker(seq, size):
    # Function to divide the list into chunks of specified size
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# Load the dataframe
# region_hovedstad_df = pd.read_csv('your_data.csv')

# Select the first 1000 rows of the dataframe
adresse_ids = DAR['id_lokalId'].iloc[800000:900000].str.strip().tolist()

df_list = []
for chunk in chunker(adresse_ids, 20):
    adresse_ids_str = '|'.join(chunk)
    response_data = get_data_from_api(adresse_ids_str)
    if response_data and 'features' in response_data:
        for feature in response_data['features']:
            properties = feature['properties']
            df_list.append(properties)

# Convert the list of dictionaries to a DataFrame
bfe_df = pd.DataFrame(df_list)
print(len(bfe_df))

# save to pickle file
bfe_df.to_pickle(r"C:\Users\viet-intel\boligpriser\data\output\EBR\EBR_800k_900k.pkl")